In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
import numpy as np
import pandas as pd

from sklearn import metrics, preprocessing


In [4]:
!ls data/input

sample_submission.csv test.csv              train.csv


In [5]:
train = pd.read_csv('data/input/train.csv')
test = pd.read_csv('data/input/test.csv')
sample_submission = pd.read_csv('data/input/sample_submission.csv')

In [6]:
train.shape
train.head()

(600000, 25)

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [7]:
train.dtypes
train.dtypes.value_counts()

id          int64
bin_0     float64
bin_1     float64
bin_2     float64
bin_3      object
bin_4      object
nom_0      object
nom_1      object
nom_2      object
nom_3      object
nom_4      object
nom_5      object
nom_6      object
nom_7      object
nom_8      object
nom_9      object
ord_0     float64
ord_1      object
ord_2      object
ord_3      object
ord_4      object
ord_5      object
day       float64
month     float64
target      int64
dtype: object

object     17
float64     6
int64       2
Name: count, dtype: int64

In [8]:
train.target.value_counts()

target
0    487677
1    112323
Name: count, dtype: int64

## Select categorical columns

In [9]:
categorical_columns = list(train.select_dtypes(include='object').columns.values)
len(categorical_columns)
categorical_columns


17

['bin_3',
 'bin_4',
 'nom_0',
 'nom_1',
 'nom_2',
 'nom_3',
 'nom_4',
 'nom_5',
 'nom_6',
 'nom_7',
 'nom_8',
 'nom_9',
 'ord_1',
 'ord_2',
 'ord_3',
 'ord_4',
 'ord_5']

In [10]:
include_columns = categorical_columns + ['target']
include_columns

['bin_3',
 'bin_4',
 'nom_0',
 'nom_1',
 'nom_2',
 'nom_3',
 'nom_4',
 'nom_5',
 'nom_6',
 'nom_7',
 'nom_8',
 'nom_9',
 'ord_1',
 'ord_2',
 'ord_3',
 'ord_4',
 'ord_5',
 'target']

In [11]:
train_categorical_df = train.loc[:, include_columns]
train_categorical_df.head()
train_categorical_df.shape

,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_1,ord_2,ord_3,ord_4,ord_5,target
0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,Contributor,Hot,c,U,Pw,0
1,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,Grandmaster,Warm,e,X,pE,0
2,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,NaN,Freezing,n,P,eN,0
3,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,Novice,Lava Hot,a,C,NaN,0
4,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,Grandmaster,Cold,h,C,OZ,0


(600000, 18)

In [12]:
train_categorical_df.isna().sum()

bin_3     18014
bin_4     18047
nom_0     18252
nom_1     18156
nom_2     18035
nom_3     18121
nom_4     18035
nom_5     17778
nom_6     18131
nom_7     18003
nom_8     17755
nom_9     18073
ord_1     18041
ord_2     18075
ord_3     17916
ord_4     17930
ord_5     17713
target        0
dtype: int64

In [13]:
test['target'] = -1
test_categorical_df = test.loc[:, include_columns]
test_categorical_df.head()
test_categorical_df.shape

,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_1,ord_2,ord_3,ord_4,ord_5,target
0,F,Y,Blue,Polygon,Axolotl,Finland,Piano,52f6dd16c,147d704e4,8d857a0a1,ca9ad1d4b,fced9e114,Novice,Boiling Hot,f,U,oU,-1
1,F,Y,Red,Circle,Lion,Russia,Bassoon,691ebeae8,8653dcc2e,67a8d4ebb,060a21580,7ca8775da,Novice,Cold,n,N,NaN,-1
2,F,Y,Blue,Circle,Axolotl,Russia,Theremin,81f792c16,6cdda499e,69403e18c,165e81a00,5940334c9,Expert,Warm,i,N,DN,-1
3,F,N,Red,Polygon,Axolotl,Costa Rica,Bassoon,c9134205b,acbca4827,cb681246b,77d41330d,6fbdeefc8,Expert,Hot,m,B,AG,-1
4,F,Y,Red,Circle,NaN,Finland,Theremin,f0f100f57,6f800b9af,cd9feb5c6,2218d9dfe,2a27c8fde,Contributor,Lava Hot,o,J,DT,-1


(400000, 18)

In [14]:
data = pd.concat([train_categorical_df ,test_categorical_df])
data.shape
data.sample(10)

(1000000, 18)

,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_1,ord_2,ord_3,ord_4,ord_5,target
159109,F,N,Green,Triangle,Hamster,India,Theremin,05eecb19a,1df72d63f,67a8d4ebb,ca9ad1d4b,3bc419b1d,Novice,Hot,n,T,JT,0
52751,T,N,Red,Polygon,Axolotl,Finland,Theremin,8c5bd88e0,6708e676d,a306b1f4e,d7e75499d,c2bd37d86,Expert,Hot,k,NaN,IA,1
110526,T,N,Red,Triangle,Lion,India,Theremin,2c39d8ba3,81a4b3fd7,e4340c326,ad5be5be4,602d7a9af,NaN,Warm,n,M,UV,-1
360837,F,N,Blue,Trapezoid,Axolotl,Costa Rica,Oboe,8af144866,e49e58b2d,728490578,c2036219b,5704a43ba,Novice,Boiling Hot,n,M,TP,1
259665,F,Y,Blue,Circle,Hamster,China,Bassoon,187c74f5c,89442059b,83bdea3a5,d8e02864a,be76e74af,Contributor,Warm,h,H,LS,-1
436570,F,NaN,Green,Polygon,Hamster,Russia,Oboe,596e36e7d,9a7ce5b05,216083343,31db9b004,7c805bba8,Expert,Lava Hot,g,C,ja,0
272426,F,Y,Blue,Circle,Dog,Costa Rica,Bassoon,b574c9841,239707c71,8da514edb,750cb6661,NaN,Novice,Cold,a,T,CN,0
64564,F,N,Blue,Polygon,Axolotl,India,Oboe,6437de06d,dde11a3cc,7bf938847,32412c81c,e4bf32721,Novice,Lava Hot,g,C,uW,-1
325780,F,N,Blue,Trapezoid,Dog,Russia,Theremin,9b70ac13e,5f98e802b,22a1b223f,fbbafaadc,162e8e7a2,Grandmaster,Warm,h,U,us,-1
480738,T,Y,Red,Polygon,Axolotl,Costa Rica,Theremin,14039fa0e,336ebc4f3,d40ca0718,fbbafaadc,e02d06c48,Contributor,Boiling Hot,h,Q,kv,1


In [15]:
for cat_col in categorical_columns:
    label_encoder = preprocessing.LabelEncoder()
    data[cat_col] = label_encoder.fit_transform(data[cat_col])

In [16]:
data.head()

,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_1,ord_2,ord_3,ord_4,ord_5,target
0,0,0,2,4,3,5,0,1059,1013,87,1,27,0,3,2,20,56,0
1,0,1,2,3,0,6,3,210,359,27,69,2112,2,5,4,23,150,0
2,0,0,2,6,3,0,0,860,694,90,102,2218,5,2,13,15,105,0
3,0,0,2,0,3,3,3,477,241,51,170,2167,4,4,0,2,190,0
4,1,0,2,5,3,2,4,556,361,182,222,1747,2,1,7,2,50,0


In [17]:
# train_categorical_df.target==1

In [18]:
train_categorical_df.head()

,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_1,ord_2,ord_3,ord_4,ord_5,target
0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,Contributor,Hot,c,U,Pw,0
1,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,Grandmaster,Warm,e,X,pE,0
2,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,NaN,Freezing,n,P,eN,0
3,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,Novice,Lava Hot,a,C,NaN,0
4,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,Grandmaster,Cold,h,C,OZ,0


In [19]:
# train_categorical_df.loc[:, train_categorical_df.target==1]

In [20]:
train_categorical_df = data.loc[data.target!=-1, :]
test_categorical_df = data.loc[data.target==-1, :]

In [21]:
train_categorical_df.shape
train_categorical_df.head()

test_categorical_df.shape
test_categorical_df.head()

(600000, 18)

,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_1,ord_2,ord_3,ord_4,ord_5,target
0,0,0,2,4,3,5,0,1059,1013,87,1,27,0,3,2,20,56,0
1,0,1,2,3,0,6,3,210,359,27,69,2112,2,5,4,23,150,0
2,0,0,2,6,3,0,0,860,694,90,102,2218,5,2,13,15,105,0
3,0,0,2,0,3,3,3,477,241,51,170,2167,4,4,0,2,190,0
4,1,0,2,5,3,2,4,556,361,182,222,1747,2,1,7,2,50,0


(400000, 18)

,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_1,ord_2,ord_3,ord_4,ord_5,target
0,0,1,0,1,0,3,2,372,147,140,174,2196,4,0,5,20,146,-1
1,0,1,2,0,4,5,0,484,837,97,4,1107,4,1,13,13,190,-1
2,0,1,0,0,0,5,3,611,676,101,16,812,1,5,8,13,12,-1
3,0,0,2,1,0,2,0,966,1048,189,107,996,1,3,12,1,0,-1
4,0,1,2,0,6,3,3,1156,685,190,30,371,0,4,14,9,14,-1


In [22]:
categorical_columns

['bin_3',
 'bin_4',
 'nom_0',
 'nom_1',
 'nom_2',
 'nom_3',
 'nom_4',
 'nom_5',
 'nom_6',
 'nom_7',
 'nom_8',
 'nom_9',
 'ord_1',
 'ord_2',
 'ord_3',
 'ord_4',
 'ord_5']

## Test on fewer categorical columns

In [23]:
train_catg_df = train_categorical_df.loc[:, ['bin_3', 'bin_4', 'target']]
test_catg_df = test_categorical_df.loc[:, ['bin_3', 'bin_4', 'target']]

train_catg_df.head()
test_catg_df.head()

,bin_3,bin_4,target
0,0,0,0
1,0,1,0
2,0,0,0
3,0,0,0
4,1,0,0


,bin_3,bin_4,target
0,0,1,-1
1,0,1,-1
2,0,1,-1
3,0,0,-1
4,0,1,-1


In [28]:
train_catg_df['bin_3'].value_counts()
train_catg_df['bin_4'].value_counts()

train_catg_df['target'].value_counts()

bin_3
0    366212
1    215774
2     18014
Name: count, dtype: int64

bin_4
0    312344
1    269609
2     18047
Name: count, dtype: int64

target
0    487677
1    112323
Name: count, dtype: int64

In [29]:
num_categorical_levels = {}
for column in ['bin_3', 'bin_4']:
    num_categorical_levels[column] = train_catg_df[column].nunique()

num_categorical_levels

{'bin_3': 3, 'bin_4': 3}

In [30]:
class CategoricalEncodingModel(nn.Module):
    def __init__(self, num_categorical_levels, embedding_dim):
        super(CategoricalEncodingModel, self).__init__()
        self.embeddings_bin3 = nn.Embedding(num_categorical_levels['bin_3'], embedding_dim)
        self.embeddings_bin4 = nn.Embedding(num_categorical_levels['bin_4'], embedding_dim)
        self.num_embs = 2*embedding_dim

        self.linear1 = nn.Linear(in_features=self.num_embs, out_features=32)
        self.linear2 = nn.Linear(in_features=32, out_features=1)

    def forward(self, cat_data_input):
        embs1 = self.embeddings_bin3(cat_data_input[0]).view(1, -1)
        embs2 = self.embeddings_bin4(cat_data_input[1]).view(1, -1)
        embs_cat = torch.cat((embs1, embs2), 1)
        out = self.linear1(embs_cat)
        out = self.linear2(out)

        return out

In [31]:
catg_enc_model = CategoricalEncodingModel(num_categorical_levels=num_categorical_levels, embedding_dim=10)
catg_enc_model

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

In [33]:
train_catg_df.head()
example_cat_data_input = torch.tensor([0, 0], dtype=torch.long)
example_cat_data_input

,bin_3,bin_4,target
0,0,0,0
1,0,1,0
2,0,0,0
3,0,0,0
4,1,0,0


tensor([0, 0])

In [34]:
logits = catg_enc_model(example_cat_data_input)
logits

tensor([[-0.0069]], grad_fn=<AddmmBackward0>)

In [37]:
catg_enc_model.state_dict()

OrderedDict([('embeddings_bin3.weight',
              tensor([[-8.1448e-01, -9.1669e-01,  5.4252e-01,  1.3378e+00, -7.1273e-02,
                       -1.6469e+00,  8.4098e-01, -3.1005e-01,  3.3774e-01, -6.9402e-01],
                      [-3.8391e-01,  3.4689e-01,  2.5516e+00, -8.0720e-01, -5.7014e-01,
                        5.3846e-01,  9.1875e-01, -1.1020e+00,  1.5297e+00, -2.6117e-01],
                      [-6.1215e-04,  1.5526e+00,  7.9828e-01, -1.8805e-01,  9.7693e-02,
                        9.9620e-01, -1.3022e+00, -1.2149e+00,  1.7847e-01,  6.9572e-01]])),
             ('embeddings_bin4.weight',
              tensor([[-1.0255,  1.1686, -0.3779,  0.0142, -0.6414,  0.8768,  1.4545,  0.0372,
                        0.5598,  0.9986],
                      [ 0.1861,  0.4124, -0.9746,  0.4944,  0.6995, -0.6605,  0.5686,  0.3725,
                        0.4852,  0.0848],
                      [ 0.2620, -0.9102,  1.0681, -0.9493, -0.7222, -0.3138,  1.2276, -0.5929,
                 

In [36]:
list(catg_enc_model.parameters())

[Parameter containing:
 tensor([[-8.1448e-01, -9.1669e-01,  5.4252e-01,  1.3378e+00, -7.1273e-02,
          -1.6469e+00,  8.4098e-01, -3.1005e-01,  3.3774e-01, -6.9402e-01],
         [-3.8391e-01,  3.4689e-01,  2.5516e+00, -8.0720e-01, -5.7014e-01,
           5.3846e-01,  9.1875e-01, -1.1020e+00,  1.5297e+00, -2.6117e-01],
         [-6.1215e-04,  1.5526e+00,  7.9828e-01, -1.8805e-01,  9.7693e-02,
           9.9620e-01, -1.3022e+00, -1.2149e+00,  1.7847e-01,  6.9572e-01]],
        requires_grad=True),
 Parameter containing:
 tensor([[-1.0255,  1.1686, -0.3779,  0.0142, -0.6414,  0.8768,  1.4545,  0.0372,
           0.5598,  0.9986],
         [ 0.1861,  0.4124, -0.9746,  0.4944,  0.6995, -0.6605,  0.5686,  0.3725,
           0.4852,  0.0848],
         [ 0.2620, -0.9102,  1.0681, -0.9493, -0.7222, -0.3138,  1.2276, -0.5929,
          -0.3364, -0.0321]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.0943,  0.1265,  0.1711,  0.1196,  0.0954, -0.0849, -0.1247, -0.1060,
           

In [38]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(catg_enc_model.parameters(), lr=0.01)

In [39]:
train_catg_df.target.value_counts()

target
0    487677
1    112323
Name: count, dtype: int64

In [42]:
sample_data = train_catg_df.sample(10, random_state=43)
sample_data

,bin_3,bin_4,target
99666,0,2,0
159761,0,1,1
105919,0,0,0
188760,0,1,0
464731,0,1,0
178950,1,1,0
504456,0,0,0
291511,0,0,0
526743,1,0,0
98102,0,0,0


In [45]:
catg_enc_model

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

In [52]:
loss_values = []
for epoch in range(5):
    total_loss = 0
    for input_x1, input_x2, y in sample_data.values:
        # print(input_x1, input_x2, y)
        cat_cols_input = torch.tensor([input_x1, input_x2], dtype=torch.long)
        target = torch.tensor(y, dtype=torch.float)

        # model training mode
        catg_enc_model.train()
        # logits = catg_enc_model(cat_cols_input)  # shape will be [1, 1] so need to squeeze 
        # print(f'logits.shape: {logits.shape}')

        logits = catg_enc_model(cat_cols_input).squeeze()
        # print(f'logits: {logits}')
        loss = loss_function(logits, target)

        # backpropagation
        loss.backward()

        # update model parameters
        optimizer.step()

        total_loss += loss
    loss_values.append(total_loss)

print(f'loss_values: {loss_values}')

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

...

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

CategoricalEncodingModel(
  (embeddings_bin3): Embedding(3, 10)
  (embeddings_bin4): Embedding(3, 10)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

loss_values: [tensor(3.3754, grad_fn=<AddBackward0>), tensor(3.6834, grad_fn=<AddBackward0>), tensor(2.4904, grad_fn=<AddBackward0>), tensor(2.9169, grad_fn=<AddBackward0>), tensor(2.7318, grad_fn=<AddBackward0>)]


In [53]:
catg_enc_model.embeddings_bin3.weight

Parameter containing:
tensor([[ 1.2500e+00,  1.5430e+00, -5.9764e-01, -2.0758e+00, -1.4551e-01,
          3.3795e+00, -1.3366e+00,  2.9048e-01, -3.5560e-01,  1.2215e+00],
        [-3.3297e-01,  5.4438e-01,  3.0106e+00, -1.1280e+00, -5.7875e-01,
          1.0383e-01,  9.7325e-01, -1.2814e+00,  1.8011e+00, -2.8478e-01],
        [-6.1215e-04,  1.5526e+00,  7.9828e-01, -1.8805e-01,  9.7693e-02,
          9.9620e-01, -1.3022e+00, -1.2149e+00,  1.7847e-01,  6.9572e-01]],
       requires_grad=True)

In [54]:
catg_enc_model.embeddings_bin4.weight

Parameter containing:
tensor([[-1.2272,  1.1315, -0.4879, -0.6067, -1.1531,  1.3928,  1.9960, -0.1623,
         -0.1877,  1.3936],
        [-0.1209, -0.7328,  1.3664, -0.4009, -0.6865,  0.6415, -1.0096, -0.2731,
         -0.1737, -0.2357],
        [ 0.1192, -0.8627,  1.0053, -1.2238, -0.9756, -0.0328,  1.5227, -0.6736,
         -0.6130,  0.2028]], requires_grad=True)